# Load Package

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

import os
cur_path = "/content/drive/MyDrive/Bigdataproject/"
os.chdir(cur_path)
!pwd

/content/drive/MyDrive/Bigdataproject


In [3]:
!pip install pyspark
!pip install pyspark[sql]
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u362-ga-0ubuntu1~20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [4]:
import pyspark as spark
import time
from operator import add
from pyspark.sql import SparkSession

In [5]:
import numpy as np
import random

# Step 0: Read Data

## Set spark session

In [6]:
# set spark session
ss = (SparkSession
  .builder
  .master("local[5]")
  .appName("fqproject")
  .getOrCreate())


## Set spark context

In [7]:
# set spark session
sc = (spark
  .SparkContext
  .getOrCreate(spark
    .SparkConf()
    .setAppName("fqproject")
    .setMaster('local[*]')
    .set('spark.executor.memory', '4G')
    .set('spark.driver.memory', '4G')
    .set('spark.driver.maxResultSize', '4G')))

## Read reads.fq

In [8]:
# read reads.fq and extract the second line of every four lines.
reads = (sc
  .textFile(cur_path + "data/reads.fq")
  .zipWithIndex()
  .filter(lambda x: (x[1]+1)%2==0 and (x[1]+1)%4!=0)
  .map(lambda x: x[0]))

In [9]:
reads.take(10)

['TCCTTACTGGTTTTGCAGGTAACTTATAGAGTATTTCCACTTCCCTTCTCCTATCCCTTGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATC',
 'GGTTTTTCAGGTAACTTATAGAGTATTTCCACTTCCCTTCTCCTATCCCTGGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATCAAAGAATA',
 'AGGTAACTTATAGAGTACTTCCACTTCCCTTCTCCTATCCCTTGAAAAATTGTCATTGATTTCTCTTATCCATATGGCATAATCAAAGAATAAATTGGTG',
 'CACTTCCCTTCTACTATCCCTTGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATG',
 'CCTTCTCCTATCCCTTGAAAAATTGTCATTTATTTCCCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTTTT',
 'ATCCCTTGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCGTGCCTATTAGATTCATT',
 'TCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTATTAGATTCATTTAGAATATAAAAAAATTTTATTTTATT',
 'TATCCATATGGCATAATCAAAGAATCAATTGTTGATATTTGTTCAAAAATCCATGCCTATTAGATTCATTTAGAATATAAAAAAATTTTATTTTATTTTC',
 'TCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTATTAGATTCATTTAGATTATAAAAAAATTTTATTTTATTTTCACTTATTTCTTCTCCA',
 'TGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTACTAGATTCATTTAG

## Read reference.fa

In [10]:
# read reference
reference = (ss
  .read
  .csv(
      cur_path + "data/reference_chr21_20000000_20050000.fa",
      inferSchema=True,
      header=True))

In [11]:
reference.show(10)

+--------+--------+--------------------+
|   start|     end|            sequence|
+--------+--------+--------------------+
|20000000|20000100|CCCTTCTCCTATCCCTT...|
|20000100|20000200|TAGATTCATTTAGAATA...|
|20000200|20000300|TTCTTCTTCCTGAAGAA...|
|20000300|20000400|ACTTTTCAAGGATAGTT...|
|20000400|20000500|CTGACAGGACTTCTGCC...|
|20000500|20000600|TTTCCTTTTTTTTTTCT...|
|20000600|20000700|ATTATAAAAAGGGAGGG...|
|20000700|20000800|TTCTTTTCTTTTTCTCC...|
|20000800|20000900|ATAAATTTCTGCTTGAA...|
|20000900|20001000|TTCGTTAGTGTTTTTTA...|
+--------+--------+--------------------+
only showing top 10 rows



In [12]:
reference.count()

500

# Step 1: Cut reads into kmers = 15

## Define K-mers

In [13]:
# input a string s and cut k-mers
def extract_kmers(s: str, k: int = 15) -> list:
    return [s[i:i+k] for i in range(len(s)-k+1)]

## Map K-mers to reads

In [14]:
# K-mers of reads
reads_kmers = (reads
  .map(extract_kmers))

In [15]:
reads_kmers.take(2)

[['TCCTTACTGGTTTTG',
  'CCTTACTGGTTTTGC',
  'CTTACTGGTTTTGCA',
  'TTACTGGTTTTGCAG',
  'TACTGGTTTTGCAGG',
  'ACTGGTTTTGCAGGT',
  'CTGGTTTTGCAGGTA',
  'TGGTTTTGCAGGTAA',
  'GGTTTTGCAGGTAAC',
  'GTTTTGCAGGTAACT',
  'TTTTGCAGGTAACTT',
  'TTTGCAGGTAACTTA',
  'TTGCAGGTAACTTAT',
  'TGCAGGTAACTTATA',
  'GCAGGTAACTTATAG',
  'CAGGTAACTTATAGA',
  'AGGTAACTTATAGAG',
  'GGTAACTTATAGAGT',
  'GTAACTTATAGAGTA',
  'TAACTTATAGAGTAT',
  'AACTTATAGAGTATT',
  'ACTTATAGAGTATTT',
  'CTTATAGAGTATTTC',
  'TTATAGAGTATTTCC',
  'TATAGAGTATTTCCA',
  'ATAGAGTATTTCCAC',
  'TAGAGTATTTCCACT',
  'AGAGTATTTCCACTT',
  'GAGTATTTCCACTTC',
  'AGTATTTCCACTTCC',
  'GTATTTCCACTTCCC',
  'TATTTCCACTTCCCT',
  'ATTTCCACTTCCCTT',
  'TTTCCACTTCCCTTC',
  'TTCCACTTCCCTTCT',
  'TCCACTTCCCTTCTC',
  'CCACTTCCCTTCTCC',
  'CACTTCCCTTCTCCT',
  'ACTTCCCTTCTCCTA',
  'CTTCCCTTCTCCTAT',
  'TTCCCTTCTCCTATC',
  'TCCCTTCTCCTATCC',
  'CCCTTCTCCTATCCC',
  'CCTTCTCCTATCCCT',
  'CTTCTCCTATCCCTT',
  'TTCTCCTATCCCTTG',
  'TCTCCTATCCCTTGA',
  'CTCCTATCCC

# Step 2: Cut reference bins into kmers = 15

In [16]:
# K-mers of reference
reference_kmers = (reference
  .select('sequence')
  .rdd
  .flatMap(lambda x: x)
  .map(extract_kmers))

In [17]:
reference_kmers.take(2)

[['CCCTTCTCCTATCCC',
  'CCTTCTCCTATCCCT',
  'CTTCTCCTATCCCTT',
  'TTCTCCTATCCCTTG',
  'TCTCCTATCCCTTGA',
  'CTCCTATCCCTTGAA',
  'TCCTATCCCTTGAAA',
  'CCTATCCCTTGAAAA',
  'CTATCCCTTGAAAAA',
  'TATCCCTTGAAAAAT',
  'ATCCCTTGAAAAATT',
  'TCCCTTGAAAAATTG',
  'CCCTTGAAAAATTGT',
  'CCTTGAAAAATTGTC',
  'CTTGAAAAATTGTCA',
  'TTGAAAAATTGTCAT',
  'TGAAAAATTGTCATT',
  'GAAAAATTGTCATTT',
  'AAAAATTGTCATTTA',
  'AAAATTGTCATTTAT',
  'AAATTGTCATTTATT',
  'AATTGTCATTTATTT',
  'ATTGTCATTTATTTC',
  'TTGTCATTTATTTCT',
  'TGTCATTTATTTCTC',
  'GTCATTTATTTCTCT',
  'TCATTTATTTCTCTT',
  'CATTTATTTCTCTTA',
  'ATTTATTTCTCTTAT',
  'TTTATTTCTCTTATC',
  'TTATTTCTCTTATCC',
  'TATTTCTCTTATCCA',
  'ATTTCTCTTATCCAT',
  'TTTCTCTTATCCATA',
  'TTCTCTTATCCATAT',
  'TCTCTTATCCATATG',
  'CTCTTATCCATATGG',
  'TCTTATCCATATGGC',
  'CTTATCCATATGGCA',
  'TTATCCATATGGCAT',
  'TATCCATATGGCATA',
  'ATCCATATGGCATAA',
  'TCCATATGGCATAAT',
  'CCATATGGCATAATC',
  'CATATGGCATAATCA',
  'ATATGGCATAATCAA',
  'TATGGCATAATCAAA',
  'ATGGCATAAT

# Step 3: Collect all kmers and build a distinct kmer set

• Hint: you can use python “set” function

• Report the number of distinct kmers (N)

In [18]:
# Merge and distinct, collect to a set
kmers = set(reads_kmers 
  .union(reference_kmers)
  .flatMap(lambda x: x) 
  .distinct()
  .collect())

In [19]:
len(kmers)

72530

**The number of distinct kmers is 72,530**


In [20]:
reads

PythonRDD[40] at RDD at PythonRDD.scala:53

# Step4

In [21]:
from pyspark.ml.linalg import Vectors
import pandas as pd

In [22]:

# turn reads to list 
r_list = reads.collect()

In [23]:
# check feature vector of every kmer in each read
def check_kmer_features(kmer):
  m = []
  

  for i in range(len(r_list)):
    if kmer in r_list[i]:
      m.append(i)
  v = [1.0 for i in range(len(m))]
  return (kmer,Vectors.dense(Vectors.sparse(len(r_list),m,v)))

In [24]:
kmers_rdd = sc.parallelize(list(kmers))

In [25]:
reads_features = kmers_rdd.map(lambda x: check_kmer_features(x))

In [26]:
reads_features_df = reads_features.toDF(['kmer','read_features'])

In [27]:
#generate the dataframe with sparse vector
reads_features_df.show(1)

+---------------+--------------------+
|           kmer|       read_features|
+---------------+--------------------+
|TTGCCCATCAAACTG|[0.0,0.0,0.0,0.0,...|
+---------------+--------------------+
only showing top 1 row



In [28]:
from pyspark.sql.functions import col
from pyspark.ml.functions import vector_to_array

In [29]:
reads_features_df = reads_features_df.withColumn("read", vector_to_array("read_features")).select(["kmer"] + [col("read")[i] for i in range(2000)])

In [30]:
read_name = {r_list[i]: 'reads' + str(i) for i in range(len(r_list))}

In [31]:
# old code
def check_kmer_features_old1(read):
  m = []
  kmers_list = list(kmers)
  for i in range(len(kmers_list)):
    if kmers_list[i] in read:
      m.append(i)
  v = [1.0 for i in range(len(m))]
  return (read_name[read],Vectors.sparse(len(kmers_list),m,v))


In [32]:
reads_features = reads.map(lambda x: check_kmer_features_old1(x))

In [33]:
reads_features.take(2)

[('reads0',
  SparseVector(72530, {173: 1.0, 854: 1.0, 1697: 1.0, 2461: 1.0, 4007: 1.0, 4617: 1.0, 4890: 1.0, 6086: 1.0, 6582: 1.0, 7639: 1.0, 7934: 1.0, 9009: 1.0, 11641: 1.0, 13192: 1.0, 16383: 1.0, 17643: 1.0, 18178: 1.0, 18285: 1.0, 19249: 1.0, 19561: 1.0, 20322: 1.0, 20401: 1.0, 21796: 1.0, 22094: 1.0, 23404: 1.0, 23748: 1.0, 23945: 1.0, 23968: 1.0, 24726: 1.0, 25532: 1.0, 25950: 1.0, 26530: 1.0, 26769: 1.0, 27986: 1.0, 28203: 1.0, 30103: 1.0, 31467: 1.0, 34054: 1.0, 34664: 1.0, 35094: 1.0, 36087: 1.0, 37854: 1.0, 38459: 1.0, 40043: 1.0, 40470: 1.0, 40760: 1.0, 41115: 1.0, 41686: 1.0, 42491: 1.0, 42563: 1.0, 42907: 1.0, 43062: 1.0, 43484: 1.0, 44210: 1.0, 45094: 1.0, 46237: 1.0, 50527: 1.0, 51926: 1.0, 53794: 1.0, 54050: 1.0, 54199: 1.0, 54924: 1.0, 55900: 1.0, 56228: 1.0, 58228: 1.0, 59435: 1.0, 59630: 1.0, 61056: 1.0, 61063: 1.0, 61217: 1.0, 61339: 1.0, 61470: 1.0, 61888: 1.0, 61979: 1.0, 62583: 1.0, 63355: 1.0, 63447: 1.0, 65004: 1.0, 65355: 1.0, 65726: 1.0, 66321: 1.0, 67214: 

# Step5

In [34]:
reference_rdd = reference.select("sequence").rdd.map(lambda x: x['sequence'])

In [35]:
reference_list = reference_rdd.collect()
reference_list[0]

'CCCTTCTCCTATCCCTTGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTAT'

In [36]:
#generate the name of columns
start_rdd = reference.select("start").rdd.map(lambda x: x['start'])
end_rdd = reference.select("end").rdd.map(lambda x: x['end'])
s_list = start_rdd.collect()
e_list = end_rdd.collect()
reference_name = ['bin' + str(s_list[i]) + '_' +  str(e_list[i]) for i in range(len(s_list))]
reference_name[0]

'bin20000000_20000100'

In [37]:
# check feature vector of every kmer in each reference
def check_kmer_read_features_sequence(kmer):
  m = []
  for i in range(len(reference_list)):
    if kmer in reference_list[i]:
      m.append(i)
  v = [1.0 for i in range(len(m))]
  return (kmer,Vectors.dense(Vectors.sparse(len(reference_list),m,v)))

In [38]:
reference_features = kmers_rdd.map(lambda x:check_kmer_read_features_sequence(x))

In [39]:
# turn to dataframe
reference_features_df = reference_features.toDF(['kmer','reference_features'])

In [40]:
reference_features_df.show(1) 

+---------------+--------------------+
|           kmer|  reference_features|
+---------------+--------------------+
|TTGCCCATCAAACTG|[0.0,0.0,0.0,0.0,...|
+---------------+--------------------+
only showing top 1 row



In [41]:
# expand the dense vector to different collomns
reference_features_df = reference_features_df.withColumn("xs", vector_to_array("reference_features")).select(["kmer"] + [col("xs")[i] for i in range(len(reference_name))])

In [42]:
reference_name = ['kmer'] + reference_name
reference_features_df = reference_features_df.select([col(c).alias(reference_name[i]) for i, c in enumerate(reference_features_df.columns)])

In [43]:
reference_name_d = {reference_list[i]: reference_name[i+1] for i in range(len(reference_list))}
len(reference_name_d)

500

In [44]:
# old code
def check_kmer_features_old(read):
  m = []
  kmers_list = list(kmers)
  for i in range(len(kmers_list)):
    if kmers_list[i] in read:
      m.append(i)
  v = [1.0 for i in range(len(m))]
  return (reference_name_d[read],Vectors.sparse(len(kmers_list),m,v))

In [45]:
# old code 
reference_features = reference_rdd.map(lambda x: check_kmer_features_old(x))
reference_features.take(1)

[('bin20000000_20000100',
  SparseVector(72530, {173: 1.0, 854: 1.0, 2589: 1.0, 4999: 1.0, 6086: 1.0, 7345: 1.0, 7639: 1.0, 8864: 1.0, 9009: 1.0, 9270: 1.0, 11641: 1.0, 13474: 1.0, 13825: 1.0, 14090: 1.0, 15247: 1.0, 15323: 1.0, 15831: 1.0, 16309: 1.0, 16666: 1.0, 17215: 1.0, 17643: 1.0, 18178: 1.0, 19249: 1.0, 19561: 1.0, 20322: 1.0, 20401: 1.0, 21796: 1.0, 22094: 1.0, 23019: 1.0, 25532: 1.0, 25628: 1.0, 26032: 1.0, 26530: 1.0, 27986: 1.0, 33453: 1.0, 33761: 1.0, 34054: 1.0, 34654: 1.0, 36126: 1.0, 38459: 1.0, 38721: 1.0, 40470: 1.0, 40760: 1.0, 41115: 1.0, 41686: 1.0, 42043: 1.0, 42491: 1.0, 42563: 1.0, 43062: 1.0, 43789: 1.0, 44210: 1.0, 45094: 1.0, 46237: 1.0, 46868: 1.0, 47310: 1.0, 48107: 1.0, 49403: 1.0, 49868: 1.0, 50160: 1.0, 51926: 1.0, 53094: 1.0, 53762: 1.0, 53794: 1.0, 54050: 1.0, 54098: 1.0, 57834: 1.0, 58228: 1.0, 58709: 1.0, 59435: 1.0, 61056: 1.0, 61217: 1.0, 61979: 1.0, 62470: 1.0, 62583: 1.0, 63355: 1.0, 63447: 1.0, 65036: 1.0, 65355: 1.0, 66321: 1.0, 67539: 1.0, 675


# Step 6: Use minhash to reduce the feature dimension for reads and reference bins

In [46]:
# define minhash
def minhash(hashtable: list, entry: tuple) -> tuple:
  ans = []
  for ls in hashtable:
    for i in ls:
      if entry[1][ls[i]]==1:
        ans.append(i)
        break
  return (entry[0], ans)

In [47]:
# create hash table
def minhash_table(n: int) -> list:
  ans = []
  for i in range(n):
    ans.append(random.sample(range(72530), 72530))
  return ans

In [48]:
# set seed
random.seed(114514)

In [49]:
# create hash table
hashtable = minhash_table(1000)

In [50]:
# run minhash on reads
start = time.time()
print(time.localtime(start))
read_sig_temp = reads_features.map(lambda x: minhash(hashtable, x))
print(time.time()-start)

time.struct_time(tm_year=2023, tm_mon=4, tm_mday=11, tm_hour=1, tm_min=42, tm_sec=9, tm_wday=1, tm_yday=101, tm_isdst=0)
0.00043511390686035156


In [51]:
# run minhash on references
start = time.time()
print(time.localtime(start))
ref_sig_temp = reference_features.map(lambda x: minhash(hashtable, x))
print(time.time()-start)

time.struct_time(tm_year=2023, tm_mon=4, tm_mday=11, tm_hour=1, tm_min=42, tm_sec=9, tm_wday=1, tm_yday=101, tm_isdst=0)
0.00022935867309570312


In [52]:
len(read_sig_temp.take(1)[0][1])

1000

In [53]:
len(ref_sig_temp.take(1)[0][1])

1000

# step 7

In [58]:
read_l = read_sig_temp.collect()

In [57]:
ref_l = ref_sig_temp.collect()

In [62]:
len(read_l)

2000

In [63]:
len(ref_l)

500

In [69]:
final_read = [i[0] for i in read_l]

In [76]:
final_ref = []
final_score = []
count = 0

for i in read_l:
  count += 1
  max_score = 0
  max_ref = ''
  for j in ref_l:
    score = len([1 for e in range(1000) if i[1][e] == j[1][e]])/1000
    if score > max_score:
      max_ref = j[0]
      max_score = score
  final_ref.append(max_ref)
  final_score.append(max_score)
  if count%100 == 0:
    print(count)




100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


In [77]:
final_ref[:10]

['bin20000000_20000100',
 'bin20000000_20000100',
 'bin20000000_20000100',
 'bin20000000_20000100',
 'bin20000000_20000100',
 'bin20000000_20000100',
 'bin20000000_20000100',
 'bin20000100_20000200',
 'bin20000000_20000100',
 'bin20000100_20000200']

In [81]:
final_score[:10]

[0.348, 0.253, 0.357, 0.692, 0.672, 0.641, 0.4, 0.181, 0.232, 0.374]

In [82]:
final_df = pd.DataFrame({'read': final_read,'reference': final_ref,'score': final_score})

In [85]:
final_df.head(5)

,read,reference,score
0,reads0,bin20000000_20000100,0.348
1,reads1,bin20000000_20000100,0.253
2,reads2,bin20000000_20000100,0.357
3,reads3,bin20000000_20000100,0.692
4,reads4,bin20000000_20000100,0.672


In [87]:
final_df.to_csv(cur_path + "data/final_result.csv", header=True, index=None, sep=',')
